<a href="https://colab.research.google.com/github/Gail529/MH_streamlit/blob/main/MH_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load necessary libraries

In [1]:
!pip install vaderSentiment
!pip install afinn

     |████████████████████████████████| 125 kB 27.4 MB/s 
     |████████████████████████████████| 52 kB 1.2 MB/s 
  Created wheel for afinn: filename=afinn-0.1-py3-none-any.whl size=53448 sha256=101b6060f05f685898ecc375f41c177bdd6c621c7f9cfeb92116266d7022f53f
  Stored in directory: /root/.cache/pip/wheels/9d/16/3a/9f0953027434eab5dadf3f33ab3298fa95afa8292fcf7aba75
Successfully built afinn


In [2]:
import pandas as pd 
import numpy as np

#tweet preprocessing 
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import string
import re
from nltk.tokenize import word_tokenize,sent_tokenize,TweetTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [6]:
emo_lex=pd.read_excel('/content/NRC-Emotion-Lexicon-v0.92-In105Languages-Nov2017Translations.xlsx') # emotion lexicon 
emolex_df=emo_lex[['English (en)','Positive','Negative','Anger','Anticipation','Disgust','Fear','Joy','Sadness','Surprise','Trust']]
emotions=emolex_df.columns.drop('English (en)')
emolex_df.rename(columns={'English (en)':'word'},inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [4]:

#receive tweet
#preprocess it
#run it through lexicon
#once emotion scores are generated calc dep score
#translate emotion score to final value
#final value is then translateed to result message
#streamlit interface
#receive tweet

#lowercasing and url,punctuations and numbers removal,
def Lowercasing(words):
    string=re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "",str(words))
    word=string.lower()
    return word

#Tokenization and (@)handle extraction
def Tokenization(tweet):
    tokens=sent_tokenize(tweet)
    return tokens

#punctuations
def Punctuation_removal(tokens):
    words=[ word for word in tokens if word.isalnum()]
    return words

#stemming
def stemming(text):
    stemmer=PorterStemmer()
    for  word in text:
        stemmed_words=stemmer.stem(word)
        return stemmed_words

#stopword_removal
def remove_stopwords(words):
    stop_words=set(stopwords.words("english")) 
    result=[word for word in words if word not in stop_words ]
    return result


#lemmatization
def lemmatization(text):
    lemmatizer=WordNetLemmatizer()
    lemmatized_phrase=[]
    for word in text:
        lemmatized_word=lemmatizer.lemmatize(word)
        lemmatized_phrase.append(lemmatized_word)
    return lemmatized_phrase


def clean_tweet(tweet):
    tweet_tokens=Tokenization(tweet)
    lemmatized_tweet=lemmatization(tweet_tokens)#lemmatization
    clean_string=Lowercasing(lemmatized_tweet)#lowercasing and removing numbers
    return clean_string



In [7]:
tweets=pd.read_csv('/content/final_tweets.csv')
tweets.head()

,Unnamed: 0,Unnamed: 0.1,user,location,text,tweets
0,0,0,app_stayaway,Mumbai,People can also take advantage of your phone a...,people can also take advantage of your phone a...
1,1,1,rykorua,Italia,Christmas is a good time to start practicing g...,christmas is a good time to start practicing g...
2,2,2,allevin18,"Minneapolis, MN","""Driving in the Midst of #Depression"" https://...",driving in the midst of depression mindwand...
3,3,3,DulleyTopBooks,"North Vancouver, B.C. Canada",@BCGovNews May the Prince of Peace give us ALL...,may the prince of peace give u all peace of ou...
4,4,4,sadvibes43ver,NaN,Anyone else just pretending they’re ok?\n#sadt...,anyone else just pretending they re ok sadtw...


In [8]:
tweet=tweets.iloc[1004,5]

In [15]:
from afinn import Afinn
afinn = Afinn(language='en')
print(tweet)
def tweet_emotions(tweet):
    emo_df=pd.DataFrame(0,index=['word'],columns=emotions)
    words=word_tokenize(tweet) #the body of text for each individual tweet(row)
    for word in words:
        emo_score=emolex_df[emolex_df.word == word]   
        if emo_score.empty:
            continue
        else:
            for emotion in list(emotions):
                emo_df.at['word',emotion] += emo_score[emotion]
    
    emo_df['afinn_score'] = afinn.score(tweet)
    return emo_df


emo_df=tweet_emotions(tweet)

rq im gonna pop in here  im diagnosed depression and adhd plus anxiety and add and i take antidepressan  


In [16]:
tweet_score=emo_df.iloc[0,:].values

weight_list=[1,5,24.28,5.71,15,9.28,0.71,39.28,3.57,2.14,1]
weight_list= [float(item) for item in weight_list]
final_val = np.dot(tweet_score,weight_list)
final_val

134.4

In [ ]:

def tweet_emotions(tweet):
  emo_df=pd.DataFrame(0,index=['word','tweet','sent'],columns=emotions)
  tweet=word_tokenize(tweet) #the body of text for each individual tweet(row)
  for word in tweet:
    emo_score=emolex_df[emolex_df.word == word]   
    print(emo_score)  
    if not emo_score.empty:
        for emotion in list(emotions):
            emo_df.at['word',emotion] += emo_score[emotion]
  return emo_df

def get_emotion_score():
    weight_list=[1,5,24.28,5.71,15,9.28,0.71,39.28,3.57,2.14,1,1,5,2.5]
    weight_list= [float(item) for item in weight_list]
    final_val = np.dot(song,weight_list)
    return final_val

tweet_emotion_df=tweet_emotion(tweet)


In [ ]:
final_df=tweet_emotion(tweet_df_1,'tweets')

In [26]:
print(emo_score)
emo_df

Empty DataFrame
Columns: [word, Positive, Negative, Anger, Anticipation, Disgust, Fear, Joy, Sadness, Surprise, Trust]
Index: []


,Positive,Negative,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust
word,0,2,1,0,0,0,0,1,0,0
tweet,0,0,0,0,0,0,0,0,0,0
sent,0,0,0,0,0,0,0,0,0,0


In [ ]:
emo_df.loc['word','Negative']

Series([], Name: Negative, dtype: float64)

In [ ]:
tweet_emotion_df

,Positive,Negative,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust
word,"9698 NaN Name: Positive, dtype: float64","9698 NaN Name: Negative, dtype: float64","9698 NaN Name: Anger, dtype: float64","9698 NaN Name: Anticipation, dtype: float64","9698 NaN Name: Disgust, dtype: float64","9698 NaN Name: Fear, dtype: float64","9698 NaN Name: Joy, dtype: float64","9698 NaN Name: Sadness, dtype: float64","9698 NaN Name: Surprise, dtype: float64","9698 NaN Name: Trust, dtype: float64"
tweet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:

emolex_df=emo_lex[['English (en)','Positive','Negative','Anger','Anticipation','Disgust','Fear','Joy','Sadness','Surprise','Trust']]
emotions=emolex_df.columns.drop('English (en)')
emolex_df.rename(columns={'English (en)':'word'},inplace=True)

def tweet_emotion(tweet):
  emo_df=pd.DataFrame(columns=emotions)
  #emo_df=pd.DataFrame(0,index=new_tweet_df.index, columns=emotions)
  lemmatizer=WordNetLemmatizer()
  
  #with tqdm(total=len(list(new_tweet_df.iterrows()))) as pbar:
    #for i ,row in new_tweet_df.iterrows(): 
        #pbar.update(1) #update the progress bar
        #tweet=word_tokenize(new_tweet_df.loc[i][col]) #the body of text for each individual tweet(row)
        tweet=word_tokenize(tweet) #the body of text for each individual tweet(row)

        for word in tweet:
            emo_score=emolex_df[emolex_df.word == new]     
            if not emo_score.empty:
                for emotion in list(emotions):
                    emo_df.at[0,emotion] += emo_score[emotion]

"""
    
if __name__ == '__main__':
    st.title('A simple streamlit mental health app')
    st.write('Want to know if your tweets contain some depressive characteristics')
    st.subheader('Key in or paste one of your tweets') 
    tweet=st.text_area('') 
    prediction_btn = st.button('predict')
    if prediction_btn:

"""

# **Preprocessing**